In [4]:
import wandb
import numpy as np

# Log in to WandB
wandb.login()

# Replace with your entity, project, and run ID
entity = "nlp_ls"
projects = ["dspro2-predicting-country", "dspro2-predicting-region", "dspro2-predicting-coordinates"]
run_ids = "*:10"  # or [...]

# Value to add to the summary
summary_key = None  # "test_data_run_id"
summary_value = None  # "w1098m89"

# Push best Validation Accuracy * and Validation Distance and Validation Loss as Logs
push_best = True
force_recalculate = True

# Access the run
api = wandb.Api()
for project in projects:
    project_run_ids = run_ids
    if project_run_ids[0] != "*":
        # Get limit, if limit is set get the most recent runs
        if len(project_run_ids) > 2 and project_run_ids[1] == ":":
            limit = int(project_run_ids.split(":")[1])
        # Get all runs of the project
        project_run_ids = []
        print(f"Getting run id's for project {project}")
        for run in api.runs(f"{entity}/{project}") if not limit else api.runs(f"{entity}/{project}")[:limit]:
            project_run_ids.append(run.id)
        print(f"Found {len(project_run_ids)} runs")
    for run_id in project_run_ids:
        print(f"Checking run {run_id}")
        if summary_key is not None and summary_value is not None:
            run = api.run(f"{entity}/{project}/{run_id}")

            # Update the summary
            run.summary[summary_key] = summary_value
            run.summary.update()

            print(f"Successfully updated summary of run {run_id} with {summary_key}: {summary_value}")

        # Push best Validation Accuracy * and Validation Distance and Validation Loss to summary
        # Technically, this is different from pushing it after training, because there it will use the values of the best epoch, however, this is a good approximation
        if push_best:
            run = api.run(f"{entity}/{project}/{run_id}")
            # Get all metrics in summary that start with "Validation"
            metrics = run.summary.keys()
            validation_metrics = [k for k in metrics if k.lower().startswith("validation")]
            best_validation_metrics = {}
            # For every validation metric, push the best value from the history
            for metric in validation_metrics:
                best_key = f"Best {metric}"
                if best_key in metrics and not force_recalculate:
                    continue
                all_values = run.history()[metric]  # numpy array
                all_values_without_nan = all_values[~np.isnan(all_values)]
                if len(all_values_without_nan) > 0:
                    best_value = np.max(all_values_without_nan) if "accuracy" in metric.lower() or "correct" in metric.lower() else np.min(all_values_without_nan)
                else:
                    best_value = np.nan
                best_validation_metrics[best_key] = best_value
                # Update the summary
                run.summary[best_key] = best_value
            if len(best_validation_metrics) > 0:
                run.summary.update()
                print(f"Successfully updated summary of run {run_id} with {best_validation_metrics}")
            else:
                print(f"Skipped updating summary of run {run_id}")

Getting run id's for project dspro2-predicting-country
Found 10 runs
Checking run kth52fnv
Successfully updated summary of run kth52fnv with {'Best Validation Accuracy Top 5': 0.8779091605691363, 'Best Validation Loss': 1.4306674492495821, 'Best Validation Accuracy Top 1': 0.6238261940892769, 'Best Validation Accuracy Top 3': 0.8143996874859143}
Checking run 8pqe6jmh
Successfully updated summary of run 8pqe6jmh with {'Best Validation Accuracy Top 3': 0.7875054464594258, 'Best Validation Accuracy Top 1': 0.5845816367925237, 'Best Validation Accuracy Top 5': 0.8600147242213441, 'Best Validation Loss': 1.4939452675147338}
Checking run ewnspxxj
Successfully updated summary of run ewnspxxj with {'Best Validation Accuracy Top 3': 0.7823369442733297, 'Best Validation Accuracy Top 5': 0.8547710984569618, 'Best Validation Accuracy Top 1': 0.5754616343885692, 'Best Validation Loss': 1.5119558592918878}
Checking run bc69qzqh
Successfully updated summary of run bc69qzqh with {'Best Validation Accu